In [1]:
import os 
from datasets import load_dataset
import nltk.data
import json

data_files = []
dirname = "/Users/marisa/clausal-coordinate-ellipsis/german-common-crawl/correct_files"
for filename in os.listdir(dirname):
    data_files.append(dirname + "/" + filename)

ds = load_dataset("json", data_files=data_files, split='train', streaming=True)
ds = ds.select_columns('raw_content')

In [2]:
def tokenize(text):
    sents = nltk.sent_tokenize(text['raw_content'])
    for s in sents:
        s = s + '[NEXT]'
    text['raw_content'] = str(sents)
    return text

In [3]:
ds_modified = ds.map(tokenize)
ds_modified

IterableDataset({
    features: Unknown,
    n_shards: 5
})

## Datensatz nach json schreiben

In [7]:
# all together
sent_id = 0
with open("de_de_pairs.jsonl", "a") as file:
    for tokenized_sentence in ds_modified:
        splitted = tokenized_sentence['raw_content'].split("',")
        for i in range(len(splitted)): # wenn 1. x: 3 vorne entfernen, sonst 2, am Ende immer zwei
            splitted[i] = splitted[i][2:(len(splitted[i])-1)]
            jsn = {'id:': sent_id, 'sentence': splitted[i], 'gold_sentence': splitted[i]}
            json.dumps(jsn)
            file.write(str(jsn) + '\n')
            sent_id += 1
            if sent_id % 100000 == 0:
                print(sent_id)
        if sent_id > 1000000:
            break 

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000


In [ ]:
# CHECK: json Datei schreiben mit: ID, Satz, Satz_gold (Sätze aus gesamten Datensatz) 
# TODO: neuen Datensatz aus json laden und damit Model trainieren